# DATA CLEANSING FOR SOME "CATEGORICAL" VARIABLES

> functions to replace nulls by last item:

In [1]:
# v2
# inSeries has to be inputted in ascendent order
def take_first_back_or_zero(inGroup):
    wrongData = inGroup[ np.isnan(inGroup) ]
    k = 0
    for index,value in wrongData.iteritems():
        backData = inGroup.iloc[ 0:inGroup.index.get_loc(index) ]
        if backData.empty:
            inGroup.loc[ index ] = 0
#             out[k,:] = np.array([index , 0])
        else:
            inGroup.loc[ index ] = backData.iloc[-1]
#             out[k,:] = np.array([index , backData.iloc[-1]])
#         k += 1
    return inGroup

# tardó 40mins para la variable VAR = 'mora_max'

In [2]:
def first_back_or_zero(dataframe):
    # data cleansing: null to np.nan
    dataframe[VAR] = pd.to_numeric( dataframe[VAR] , downcast = 'float' )

    # triming negative values
    dataframe[VAR] = dataframe[VAR].clip(lower = 0)

    # replacing np.nan for last value or zero
    wrongCli = dataframe[ np.isnan(df[VAR]) ]['id_cli'].unique()
    newDataVAR = dataframe[dataframe.id_cli.isin( wrongCli ) ].sort_values(by='periodo').groupby('id_cli')[VAR].apply(take_first_back_or_zero)

    # assing new values to original dataframe
    dataframe.iloc[newDataVAR.index.values , dataframe.columns.get_loc(VAR) ] = newDataVAR.values
    return dataframe

> function to convert categorical to numerical based on distance to median of gasto_familiar clustered by data of pandemia 202004:

In [3]:
def cat_to_num_by_pandemia(dataframe):
    # > prepandemia:

    medianaGF_ac = dataframe[dataframe.periodo < fecha_pandemia ].gasto_familiar.median()
    print(medianaGF_ac)

    # # getting new values for each category
    medianaVAR = dataframe[dataframe.periodo < fecha_pandemia ].groupby(VAR)['gasto_familiar'].median()
    rangoVAR =  np.abs(medianaVAR.max()) - np.abs(medianaVAR.min())
    newCat = ( medianaVAR - medianaGF_ac ) / (rangoVAR)
    print(newCat)

    # replacing new values
    ix = dataframe[ dataframe['periodo'] < (fecha_pandemia) ].index.values
    dataframe.loc[ix] = dataframe.loc[ ix ].replace( { VAR : dict(newCat) } )

    # > pospandemia:

    medianaGF_dc = dataframe[dataframe.periodo >= fecha_pandemia ].gasto_familiar.median()
    print(medianaGF_dc)

    # getting new values for each category
    medianaVAR = dataframe[dataframe.periodo >= fecha_pandemia ].groupby(VAR)['gasto_familiar'].median()
    rangoVAR =  np.abs(medianaVAR.max()) - np.abs(medianaVAR.min())
    newCat = ( medianaVAR - medianaGF_ac ) / (rangoVAR)
    print(newCat)

    # replacing new values
    ix = dataframe[ dataframe['periodo'] >= (fecha_pandemia) ].index.values
    dataframe.loc[ix] = dataframe.loc[ ix ].replace( { VAR : dict(newCat) } )
    
    return dataframe

## Reading periodo, id_cli and gasto_familiar:

In [9]:
import numpy as np
import pandas as pd

headers = pd.read_csv( './../Data/Dataton_train/header.txt' , sep= ',', header=None,encoding='utf-8')
headers = headers.values.tolist()[0]

fieldsToSelect = ['periodo', 'id_cli' ,'gasto_familiar']

df = pd.read_csv('./../Data/Dataton_train/Dataton_train_semicolon.csv',
                 header = None,  names=headers, sep=';', #iterator=True, chunksize=1000000,
                 skipinitialspace=True, usecols=fieldsToSelect, na_values='\\N')

# See the keys
print( df.info() )

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20988748 entries, 0 to 20988747
Data columns (total 3 columns):
 #   Column          Dtype  
---  ------          -----  
 0   periodo         int64  
 1   id_cli          int64  
 2   gasto_familiar  float64
dtypes: float64(1), int64(2)
memory usage: 480.4 MB
None


# > sección :
**Vamos a hacer un análisis de cada variable categórica en dos etapas: prepandemia y pospandemia:**
## (x) cat_ingreso
## (x) tipo_vivienda

In [10]:
## removing last variables

# df.drop('cat_ingreso', inplace=True, axis=1)

> reading data for specific variable:

In [11]:
import numpy as np
import pandas as pd

VAR = ['tipo_vivienda']


headers = pd.read_csv( './../Data/Dataton_train/header.txt' , sep= ',', header=None,encoding='utf-8')
headers = headers.values.tolist()[0]

df[ VAR[0] ] = pd.read_csv('./../Data/Dataton_train/Dataton_train_semicolon.csv',
                 header = None,  names=headers, sep=';', #iterator=True, chunksize=1000000,
                 skipinitialspace=True, usecols=VAR, na_values='\\N')

In [12]:
# selecting variable
VAR = VAR[0]

In [13]:
fecha_pandemia = 202004
df = cat_to_num_by_pandemia(df)
df = first_back_or_zero(df)

402000.0
tipo_vivienda
ALQUILADA     0.461319
FAMILIAR      0.490601
NO INFORMA    0.029727
PROPIA        1.029727
Name: gasto_familiar, dtype: float64
485518.41
tipo_vivienda
ALQUILADA     2.142506
FAMILIAR      2.412522
NO INFORMA    1.612806
PROPIA        2.612806
Name: gasto_familiar, dtype: float64


In [14]:
# save new data into new file
output_folder = './../Data/DataCleaned/'
compression_opts = dict(method='zip', archive_name = VAR + '.csv')  

df[VAR].to_csv( output_folder + VAR + '.zip', sep=';', header = True, index=False, compression=compression_opts) 

print('listo: {}'.format(VAR) )

listo: tipo_vivienda


In [15]:
# mora_max: TODO EMPEZÓ A LAS 1506h - 1508h (22 mins)
# cant_moras_90_ult_12_meses: TODO EMPEZÓ A LAS 1550 - 1620 (30 mins)
# cant_mora_30_tdc_ult_3m_sf: 1622h - 1640h (18mins)
# cat_ingreso: 1746h - 1850h (~1h)
# tipo_vivienda: 1854h - 
# 
# POR CADA 30% DE NULOS, TARDA 30MINS EN LLENARSE